In [1]:
import train as train
import generate
import eval
import train
from modules.config import read_config
import modules.plots as plots
from modules.datageneration import load_data, data_functions
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pyro.poutine as poutine

In [2]:
BASE_DIR = "./tests/sanity_check"
PROFILE = "sinus-(10,20)-adam"

#BASE_DIR = "./tests/x-dim-test"
#PROFILE = "test"

In [3]:
configs = read_config(f"{BASE_DIR}/config.ini")
dataset_configs = read_config(f"{BASE_DIR}/dataset_config.ini")

config = configs[PROFILE]
NAME = config["NAME"]
dataset_config = dataset_configs[config["DATASET"]]
DATASET_NAME = dataset_config["NAME"]

In [4]:
inference_model = train.make_inference_model(config, device="cpu")

AutoNormal()


In [5]:
inference_model.guide.__dict__

{'init_loc_fn': <function modules.guides.getAutoNormal.<locals>.init_func(*args)>,
 '_init_scale': 0.1,
 '_pyro_name': 'AutoNormal',
 '_pyro_context': <pyro.nn.module._Context at 0x7fcb34567d30>,
 '_pyro_params': OrderedDict(),
 '_pyro_samples': OrderedDict(),
 'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 'master': None,
 '_model': (_bound_partial(functools.partial(<function _context_wrap at 0x7fcb3445eca0>, <pyro.infer.autoguide.initialization.InitMessenger object at 0x7fcb36e3adc0>, BayesianRegressorV2(
    (fc0): PyroLinear(in_features=1, out_features=32, bias=True)
    (fc1): PyroLinear(in_features=32, out_features=1, 

In [6]:
_, _, (x_test, y_test), (x_test_in_domain, y_test_in_domain), (x_test_out_domain, y_test_out_domain) = load_data(f"{BASE_DIR}/datasets/{DATASET_NAME}", load_train=False, load_val=False)

In [7]:
with poutine.trace() as tr:
    inference_model.model(x_test[0])
for site in tr.trace.nodes.values():
    print(site["type"], site["name"], site["value"].shape)

torch.Size([1])
fc0
torch.Size([32])
fc1
torch.Size([1])
sample fc0.weight torch.Size([32, 1])
sample fc0.bias torch.Size([32])
sample fc1.weight torch.Size([1, 32])
sample fc1.bias torch.Size([1])
sample sigma torch.Size([])
sample data torch.Size([1])
sample obs torch.Size([1])


In [8]:
inference_model.model

BayesianRegressorV2(
  (fc0): PyroLinear(in_features=1, out_features=32, bias=True)
  (fc1): PyroLinear(in_features=32, out_features=1, bias=True)
  (LeakyReLU): LeakyReLU(negative_slope=0.1)
)